In [7]:
from nltk.sentiment import SentimentIntensityAnalyzer
def classify_comments(df):
    sia = SentimentIntensityAnalyzer()
    def _classify(comment):
        scores = sia.polarity_scores(comment)
        compound = scores['compound']
        if compound >= 0.05:
            return 'positive'
        elif compound <= -0.05:
            return 'negative'
        else:
            return 'neutral'
    df['sentiment'] = df['comment_column'].apply(lambda comment: _classify(comment))
    return df

In [23]:
# In the strong scaling tests, the total work for the cluster is kept constant while the number of workers increase
# The code below does 12*iterations tests. there are 3 diffrent probelm sizes for the cluster and 4 diffrent configurations of the resources.
# 1 worker 1 core
# 2 worker 2 cores
# 2 workers 1 core each
# 2 worker 2 cores each
# Given the problem size the code test all the configurations iterations time
# The data of the analysis is stored in results_df
# Imports
!pip install pandas
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, split, array_contains
import time
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
results = []
# Parameter for the run
# Data amounts to be testing
chunk_amounts = [100, 300, 600]
# Setting for 1,2 and 4 cores
executor_cores_options = [1, 2]
executor_nodes_options = [1, 2]
# Tests per above setting
iterations = 2

nltk.download('vader_lexicon')
sid = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    sentiment_score = sid.polarity_scores(text)
    return sentiment_score['compound']

sentiment_udf = udf(analyze_sentiment, FloatType())

for datasize in chunk_amounts:
   for executor_nodes in executor_nodes_options:
       for executor_cores in executor_cores_options:
           print("1")
           spark_session = SparkSession.builder\
                           .appName(f"DF_cores_{executor_cores}_instances_{executor_nodes}_datasize_{datasize*executor_nodes*executor_cores}")\
                           .master("spark://192.168.2.230:7077")\
                           .config("spark.dynamicAllocation.enabled", True)\
                           .config("spark.executor.instances", executor_nodes)\
                           .config("spark.executor.cores", executor_cores)\
                           .config("spark.dynamicAllocation.minExecutors","1")\
                           .config("spark.dynamicAllocation.maxExecutors","1")\
                           .config("spark.driver.port", 9999)\
                           .config("spark.blockManager.port", 10005)\
                           .getOrCreate()
           print("2")
           file_paths = [f"hdfs://master:9000/path/in/hdfs/JsonFiles/corpus-webis-shortened_chunk{i}.json" for i in range(1, datasize*executor_nodes*executor_cores + 1)]
           df = spark_session.read.option("multiline", "true").json(file_paths)
           for iteration in range(iterations):
                   start_time = time.time()
                   df_with_sentiment = df.withColumn("sentiment_score", sentiment_udf(df["content"]))
                   # df.show(truncate=False)
                   # df_with_sleep = df.filter(array_contains(split(lower(col("content")), " "), "sleep"))
                   # num_posts_with_sleep = df_with_sleep.count()
                   positive_comments = df_with_sentiment.filter(col("sentiment_score") < 0).count()
                   negative_comments = df_with_sentiment.filter(col("sentiment_score") > 0).count()
                   end_time = time.time()
                   processing_time = end_time - start_time
                   results.append({
                       'Workers': executor_nodes,
                       'Cores/W': executor_cores,
                       'Total Cores': executor_cores*executor_nodes,
                       'Chunks': datasize*executor_nodes*executor_cores,
                       'Iteration': iteration + 1,
                       'Time (s)': processing_time,
                       'Positive Comments': positive_comments,
                       'Negative Comments': negative_comments
                       
                   })
           print("3")
           spark_session.stop()
# Convert results to a pandas DataFrame for easy tabular display
results_df = pd.DataFrame(results)
print(results_df)

1
2


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
24/03/14 16:23:48 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


3
1
2


24/03/14 16:24:45 ERROR DAGScheduler: Failed to update accumulator 12417 (org.apache.spark.api.python.PythonAccumulatorV2) for task 1
org.apache.spark.SparkException: EOF reached before Python server acknowledged
	at org.apache.spark.api.python.PythonAccumulatorV2.merge(PythonRDD.scala:751)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1(DAGScheduler.scala:1694)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1$adapted(DAGScheduler.scala:1685)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.updateAccumulators(DAGScheduler.scala:1685)
	at org.apache.spark.scheduler.DAGScheduler.handleTaskCompletion(DAGScheduler.scala:1838)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:3054)
	

3
1
2


...

3
1
2


24/03/14 16:55:08 ERROR DAGScheduler: Failed to update accumulator 12417 (org.apache.spark.api.python.PythonAccumulatorV2) for task 1
java.net.SocketException: Broken pipe (Write failed)
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.flush(BufferedOutputStream.java:140)
	at java.io.DataOutputStream.flush(DataOutputStream.java:123)
	at org.apache.spark.api.python.PythonAccumulatorV2.merge(PythonRDD.scala:747)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1(DAGScheduler.scala:1694)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1$adapted(DAGScheduler.scala:1685)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.

3
1
2


24/03/14 17:01:54 ERROR DAGScheduler: Failed to update accumulator 12417 (org.apache.spark.api.python.PythonAccumulatorV2) for task 3
java.net.SocketException: Broken pipe (Write failed)
	at java.net.SocketOutputStream.socketWrite0(Native Method)
	at java.net.SocketOutputStream.socketWrite(SocketOutputStream.java:111)
	at java.net.SocketOutputStream.write(SocketOutputStream.java:155)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.flush(BufferedOutputStream.java:140)
	at java.io.DataOutputStream.flush(DataOutputStream.java:123)
	at org.apache.spark.api.python.PythonAccumulatorV2.merge(PythonRDD.scala:747)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1(DAGScheduler.scala:1694)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$updateAccumulators$1$adapted(DAGScheduler.scala:1685)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.

3
    Workers  Cores/W  Total Cores  Chunks  Iteration    Time (s)  \
0         1        1            1     100          1   19.914631   
1         1        1            1     100          2   17.885263   
2         1        2            2     200          1   34.861555   
3         1        2            2     200          2   32.071077   
4         2        1            2     200          1   35.712079   
5         2        1            2     200          2   32.076694   
6         2        2            4     400          1   37.165781   
7         2        2            4     400          2   34.494634   
8         1        1            1     300          1   87.949527   
9         1        1            1     300          2   85.168124   
10        1        2            2     600          1   91.449664   
11        1        2            2     600          2   91.044022   
12        2        1            2     600          1   88.649929   
13        2        1            2     600     

In [19]:
spark_session.stop()